In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
test = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])

dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])


In [2]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [3]:
# valid 제작하는 함수 새로 개편.
# 예전 valid 제작 함수는 1분 걸렸는데 0.6초만에 끗~
_train['train_valid'] = 0
_train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
_valid = _train[_train['train_valid'] == -1]
_train = _train[_train['train_valid'] == 0]

/tmp/ipykernel_86008/2300459396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _train['train_valid'] = 0


In [4]:
class elo():
    def __init__(self):
        self.item_parameters = None
        self.student_parameters = None

    def get_new_theta(self,is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + self.learning_rate_theta(nb_previous_answers) * (
            is_good_answer - self.probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(self,is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - self.learning_rate_beta(nb_previous_answers) * (
            is_good_answer - self.probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(self,nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(self,nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(self,theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * self.sigmoid(theta - beta)

    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(self,answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...", flush=True)

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            ),
            total=len(answers_df),
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = self.get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = self.get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        self.student_parameters = student_parameters
        self.item_parameters = item_parameters
        return student_parameters, item_parameters
    
    def estimate_student_parameters(self,answers_df, granularity_feature_name="assessmentItemID"):
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        #print("Parameter estimation is starting...", flush=True)

        for student_id, item_id, left_asymptote, answered_correctly in zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            ):
            theta = student_parameters[student_id]["theta"]
            beta = self.item_parameters[item_id]["beta"]

            student_parameters[student_id]["theta"] = self.get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

           
            student_parameters[student_id]["nb_answers"] += 1

        #print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        self.student_parameters = student_parameters
        return student_parameters

    def update_parameters(self,answers_df, student_parameters, item_parameters, granularity_feature_name='assessmentItemID'):
        for student_id, item_id, left_asymptote, answered_correctly in zip(
            answers_df.userID.values, 
            answers_df[granularity_feature_name].values, 
            answers_df.left_asymptote.values, 
            answers_df.answerCode.values):
            if student_id not in student_parameters:
                student_parameters[student_id] = {'theta': 0, 'nb_answers': 0}
            if item_id not in item_parameters:
                item_parameters[item_id] = {'beta': 0, 'nb_answers': 0}
                
            theta = student_parameters[student_id]['theta']
            beta = item_parameters[item_id]['beta']

            student_parameters[student_id]['theta'] = self.get_new_theta(
                answered_correctly, beta, left_asymptote, theta, student_parameters[student_id]['nb_answers']
            )
            item_parameters[item_id]['beta'] = self.get_new_beta(
                answered_correctly, beta, left_asymptote, theta, item_parameters[item_id]['nb_answers']
            )
            
            student_parameters[student_id]['nb_answers'] += 1
            item_parameters[item_id]['nb_answers'] += 1
        
        
        return student_parameters, item_parameters
    def run(self,df):
        df["left_asymptote"] = 0

        print(f"Dataset of shape {df.shape}")
        print(f"Columns are {list(df.columns)}")
        
        

        student_parameters, item_parameters = self.estimate_parameters(df,granularity_feature_name = "KnowledgeTag")


        b = [
            item_parameters[item]["beta"]
            for student, item in zip(df.userID.values, df.KnowledgeTag.values)
        ]
        

        #beta represents the difficulty of the item 
        df['elotag'] = b

        student_parameters, item_parameters = self.estimate_parameters(df,granularity_feature_name = "testId")


        b = [
            item_parameters[item]["beta"]
            for student, item in zip(df.userID.values, df.testId.values)
        ]
        

        #beta represents the difficulty of the item 
        df['elotest'] = b

        student_parameters, item_parameters = self.estimate_parameters(df)
        
        t = [
        student_parameters[student]["theta"]
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
        ]
        b = [
            item_parameters[item]["beta"]
            for student, item in zip(df.userID.values, df.assessmentItemID.values)
        ]


        #theta represents the global level of the student
        df['elouser'] = t

        #beta represents the difficulty of the item 
        df['eloitem'] = b


        return df 
    
    def run_update(self,df):

        df["left_asymptote"] = 0
        return self.estimate_student_parameters(df)[df.iloc[0,0]]['theta']


In [5]:
Elo = elo()
# ELO Function 적용
_train = Elo.run(_train)

# 필요없는 column 제거
_train = _train.drop(columns=["left_asymptote"])
 

Dataset of shape (2517453, 8)
Columns are ['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'train_valid', 'left_asymptote']
Parameter estimation is starting...


100%|██████████| 2517453/2517453 [00:18<00:00, 137080.05it/s]


Theta & beta estimations on KnowledgeTag are completed.
Parameter estimation is starting...


100%|██████████| 2517453/2517453 [00:17<00:00, 140402.67it/s]


Theta & beta estimations on testId are completed.
Parameter estimation is starting...


100%|██████████| 2517453/2517453 [00:18<00:00, 135671.27it/s]


Theta & beta estimations on assessmentItemID are completed.


In [6]:
tmp = _train.copy()
tmp['tem'] = 1
tmp['seq'] = tmp.groupby('userID')['tem'].apply(lambda x : x.cumsum())
tmp['eloupdate'] =0
dat_tem = tmp[['userID', 'assessmentItemID', 'answerCode', 'seq']]

/tmp/ipykernel_86008/2143948767.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tmp['seq'] = tmp.groupby('userID')['tem'].apply(lambda x : x.cumsum())


In [7]:
_train['eloupdate'] = tmp.apply(lambda x : Elo.run_update(dat_tem[(dat_tem['userID']==x['userID']) & (dat_tem['seq'] < x['seq']) & (dat_tem['seq'] >= x['seq']-100)]) if x['seq'] >= 100 else (Elo.run_update(dat_tem[(dat_tem['userID']==x['userID']) & (dat_tem['seq'] < x['seq'])]) if x['seq'] != 1 else 0),axis=1 )

In [ ]:
users = _train.drop_duplicates(['userID'])[['userID','elouser']]
items = _train.drop_duplicates(['assessmentItemID'])[['assessmentItemID','eloitem']]
tag = _train.drop_duplicates(['KnowledgeTag'])[['KnowledgeTag','elotag']]
testid = _train.drop_duplicates(['testId'])[['testId','elotest']]
#update = _train.drop_duplicates(['assessmentItemID'])[['assessmentItemID','eloitem']]

In [ ]:
users.to_csv('/opt/ml/input/data/FE/users_elo.csv', index=False)
items.to_csv('/opt/ml/input/data/FE/items_elo.csv', index=False)
tag.to_csv('/opt/ml/input/data/FE/tag_elo.csv', index=False)
testid.to_csv('/opt/ml/input/data/FE/testid_elo.csv', index=False)

In [ ]:
_test = pd.merge(_test, users, on='userID', how='left')
_test = pd.merge(_test, items, on='assessmentItemID', how='left')
_test = pd.merge(_test, tag, on='KnowledgeTag', how='left')
_test = pd.merge(_test, testid, on='testId', how='left')
_test

In [ ]:
_valid = pd.merge(_valid, users, on='userID', how='left')
_valid = pd.merge(_valid, items, on='assessmentItemID', how='left')
_valid = pd.merge(_test, tag, on='KnowledgeTag', how='left')
_valid = pd.merge(_test, testid, on='testId', how='left')
_valid

In [ ]:
dat = pd.concat([_train,_valid,_test])
dat = dat.sort_values(by = ['userID', 'Timestamp'])

In [ ]:
dat

In [ ]:
dat['elo'] = dat.apply(lambda x : 1 / 1 + (np.exp(-(x['elouser']-x['eloitem']))),axis=1)

In [ ]:
diff = dat.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
#diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
dat['solve_time'] = diff['Timestamp']

In [ ]:
# 시험 난이도(?)로 추청되는 특성 따로 분류
dat['b_category'] = dat['assessmentItemID'].str[2]
# 시험지로 추청되는 특성 따로 분류. 뒤에 것이 좋을 것 같은데 일단 제출을 한 것으로 써놓음.
dat['test_category'] = dat['assessmentItemID'].str[2] + dat['assessmentItemID'].str[4:7] # dat['assessmentItemID'].str[4:7]
dat['problem_id'] = dat['assessmentItemID'].str[-3:]

In [ ]:
diff['Timestamp'] = dat['solve_time']
diff['Timestamp'] = pd.qcut(diff['Timestamp'],5)
diff['Timestamp'] = diff['Timestamp'].astype("str")
dat['category_st_qcut_5'] = diff['Timestamp']

In [ ]:
dat['solve_time'] = dat['solve_time'].astype(int)

In [ ]:
dat['last_answerCode'] = dat.groupby("userID")['answerCode'].shift(1).fillna(1)
dat['last_answerCode2'] = dat.groupby("userID")['answerCode'].shift(2).fillna(1)
dat['last_answerCode3'] = dat.groupby("userID")['answerCode'].shift(3).fillna(1)
dat['last_answerCode4'] = dat.groupby("userID")['answerCode'].shift(4).fillna(1)
dat['last_answerCode5'] = dat.groupby("userID")['answerCode'].shift(5).fillna(1)
dat['last_answerCode6'] = dat.groupby("userID")['answerCode'].shift(6).fillna(1)
dat['last_answerCode7'] = dat.groupby("userID")['answerCode'].shift(7).fillna(1)
dat['last_answerCode8'] = dat.groupby("userID")['answerCode'].shift(8).fillna(1)
dat['last_answerCode9'] = dat.groupby("userID")['answerCode'].shift(9).fillna(1)
dat['last_answerCode10'] = dat.groupby("userID")['answerCode'].shift(10).fillna(1)

In [ ]:
dat['last_answerCode'] = dat['last_answerCode'].astype(int)
dat['last_answerCode2'] = dat['last_answerCode2'].astype(int)
dat['last_answerCode3'] = dat['last_answerCode3'].astype(int)
dat['last_answerCode4'] = dat['last_answerCode4'].astype(int)
dat['last_answerCode5'] = dat['last_answerCode5'].astype(int)
dat['last_answerCode6'] = dat['last_answerCode6'].astype(int)
dat['last_answerCode7'] = dat['last_answerCode7'].astype(int)
dat['last_answerCode8'] = dat['last_answerCode8'].astype(int)
dat['last_answerCode9'] = dat['last_answerCode9'].astype(int)
dat['last_answerCode10'] = dat['last_answerCode10'].astype(int)

In [ ]:
dat.drop(['train_valid'], axis = 1, inplace = True) 

In [ ]:
dat['year'] = dat['Timestamp'].dt.year # 연도 정보
dat['month'] =  dat['Timestamp'].dt.month # 월 정보
dat['day'] =  dat['Timestamp'].dt.day # 일 정보
dat['hour'] =  dat['Timestamp'].dt.hour # 시간 정보

dat.to_csv('/opt/ml/input/data/FE/FE_total_2_elo3.csv', index=False)

In [ ]:
dat['elo'].isnull().value_counts()

In [ ]:
dat.columns

In [ ]:
dat['solve_time'].value_counts()